#### Let's preprocess each column

In [3592]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import truncnorm

### Reading the excel file

In [3593]:
df = pd.read_excel("../data/processed/preprocessed_data.xlsx")

In [3594]:
df.columns

Index(['id', 'acceptance_rate', 'institution', 'program', 'degree_type',
       'decision', 'undergrad_gpa', 'gre_quantitative_reasoning',
       'gre_verbal_reasoning', 'analytical_writing', 'notes', 'gre_total'],
      dtype='object')

### Checking the maximum and minimum acceptance rate range to see whether it is valid or not.

In [3595]:
df['acceptance_rate'] = pd.to_numeric(df['acceptance_rate'].str.replace('%', ''), errors='coerce')

In [3596]:
acceptance_rate_max = df['acceptance_rate'].max(skipna=True)
acceptance_rate_min = df['acceptance_rate'].min(skipna=True)

print("Max:", acceptance_rate_max)
print("Min:", acceptance_rate_min)


Max: 100.0
Min: 6.0


### Since the range is between 6% to 100%, the acceptance rate data seems fairly valid

_____________________________________________________________________________________________________________________________________________________________________________________

### Converting the gpa to numeric values

In [3597]:
df['undergrad_gpa'] = pd.to_numeric(df['undergrad_gpa'], errors='coerce')

### Checking the range of maximum and mininum of gpa

In [3598]:
undergrad_gpa_max = df['undergrad_gpa'].max(skipna=True)
undergrad_gpa_min = df['undergrad_gpa'].min(skipna=True)

print("Max:", undergrad_gpa_max)
print("Min:", undergrad_gpa_min)


Max: 332.0
Min: 0.01


### Valid GPA case 1

In [3599]:
gpa_greater_than_equal_to_one = df['undergrad_gpa'] >= 1.0
gpa_less_than_equal_to_four = df['undergrad_gpa'] <= 4.0
valid_gpa_between_one_and_four = df[gpa_greater_than_equal_to_one & gpa_less_than_equal_to_four]
len(valid_gpa_between_one_and_four)

44138

### Valid GPA Case 2
##### Sometimes the user may enter the gpa without the '.', hence we should check if someone has entered values between 100 to 332(since 332 is the vax value)

In [3600]:
gpa_greater_than_equal_to_hundred = df['undergrad_gpa'] >= 100
gpa_less_than_equal_to_four_hundred = df['undergrad_gpa'] < 401
valid_gpa_between_one_hundred_and_below_four_hundred = df.loc[(gpa_greater_than_equal_to_hundred) & (gpa_less_than_equal_to_four_hundred)]
valid_gpa_between_one_hundred_and_below_four_hundred

,id,acceptance_rate,institution,program,degree_type,decision,undergrad_gpa,gre_quantitative_reasoning,gre_verbal_reasoning,analytical_writing,notes,gre_total
23246,965397,NaN,University of California,Informatics,PhD,Accepted,332.0,NaN,NaN,interviewed on 1/28,Acceptance,NaN


In [3601]:
condition = gpa_greater_than_equal_to_hundred & gpa_less_than_equal_to_four_hundred

In [3602]:
df.loc[condition, 'undergrad_gpa'] = df.loc[condition, 'undergrad_gpa'] / 100

In [3603]:
fresh_mask = (df['undergrad_gpa'] >= 100) & (df['undergrad_gpa'] <= 400)
len(df.loc[fresh_mask])

0

### Invalid GPA

In [3604]:
# Step 1: Identify invalid GPA entries
invalid_gpa_mask = (
    (df['undergrad_gpa'] < 1.0) |
    (df['undergrad_gpa'] > 4.0) |  # You probably meant 4.0, not 400
    (df['undergrad_gpa'].isna())
)

In [3605]:
# Step 2: Generate random GPAs in a realistic range (around 3.8)
random_gpas = np.clip(
    np.random.normal(loc=3.8, scale=0.1, size=invalid_gpa_mask.sum()),
    3.6, 4.0
)

# Step 3: Replace invalid entries with these generated values
df.loc[invalid_gpa_mask, 'undergrad_gpa'] = random_gpas

In [3606]:
df['undergrad_gpa'].isnull().sum()

0

_____________________________________________________________________________________________________________________________________________________

### The GRE General Test has three scoring sections, each with its own score range:

#### Verbal Reasoning : Score range: 130 to 170

#### Quantitative Reasoning : Score range: 130 to 170

#### Analytical Writing : Score range: 0 to 6 

In [3607]:
# Convert GRE scores to numeric
list_of_gre_columns = ['gre_quantitative_reasoning', 'gre_verbal_reasoning']
df[list_of_gre_columns[0]] = pd.to_numeric(df[list_of_gre_columns[0]], errors = 'coerce')
df[list_of_gre_columns[1]] = pd.to_numeric(df[list_of_gre_columns[1]], errors = 'coerce')

In [3608]:
gre_quantitative_reasoning_min = df[list_of_gre_columns[0]] > 129.0
gre_quantitative_reasoning_max = df[list_of_gre_columns[0]] < 171.0
valid_gre_quantitative_reasoning_profiles = df[gre_quantitative_reasoning_min & gre_quantitative_reasoning_max]
len(valid_gre_quantitative_reasoning_profiles)

10687

In [3609]:
gre_verbal_reasoning_min = df[list_of_gre_columns[1]] > 129.0
gre_verbal_reasoning_max = df[list_of_gre_columns[1]] < 171.0
valid_gre_verbal_reasoning_profiles = df[gre_verbal_reasoning_min & gre_verbal_reasoning_max]
len(valid_gre_verbal_reasoning_profiles)

11026

### Approach
Maybe a person entered their entire score instead of gre general. In that case the score boundary should be within 260 to 340. Else if it exceeds that then the same is neglected.

In [3610]:
invalid_gre_quantitative_reasoning_profiles = df[~(gre_quantitative_reasoning_min & gre_quantitative_reasoning_max) & ~df[list_of_gre_columns[0]].isna()]
len(invalid_gre_quantitative_reasoning_profiles)

1568

In [3611]:
invalid_gre_verbal_reasoning_profiles = df[~(gre_verbal_reasoning_min & gre_verbal_reasoning_max) & ~df[list_of_gre_columns[1]].isna()]
len(invalid_gre_verbal_reasoning_profiles)

68

#### Assigning values from 260 to 340 to gre_total column

In [3612]:
df['gre_total'] = df.loc[(df[list_of_gre_columns[0]] >= 260) & (df[list_of_gre_columns[0]] <= 340), list_of_gre_columns[0]]

In [3613]:
df['gre_total'] = df.loc[(df[list_of_gre_columns[1]] >= 260) & (df[list_of_gre_columns[1]] <= 340), list_of_gre_columns[1]]

#### Removing values 260 to 340 from gre_general column to gre_total column

In [3614]:
df.loc[(df[list_of_gre_columns[0]] >= 260) & (df[list_of_gre_columns[0]] <= 340), list_of_gre_columns[0]] = 0

In [3615]:
df.loc[(df[list_of_gre_columns[1]] >= 260) & (df[list_of_gre_columns[1]] <= 340), list_of_gre_columns[1]] = 0

### Re-checking if there are any values between 260 and 340

In [3616]:
len(df.loc[(df[list_of_gre_columns[0]] >= 260) & (df[list_of_gre_columns[0]] <= 340)])

0

In [3617]:
len(df.loc[(df[list_of_gre_columns[1]] >= 260) & (df[list_of_gre_columns[1]] <= 340)])

0

## Filling in empty values for GRE quantitative reasoning

### Histogram for GRE Quantitative reasoning

In [3618]:
fig = px.histogram(valid_gre_quantitative_reasoning_profiles, x=list_of_gre_columns[0], nbins=30,
                   title=f'Distribution of Valid {list_of_gre_columns[0]} Scores')
fig.show()

In [3619]:
median_val = df[list_of_gre_columns[0]].median()
mode_val = df[list_of_gre_columns[0]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[0]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 165.0, Mode: 170.0, Mean: 144.23215014279884


In [ ]:
# import numpy as np

# # Step 1: Count missing
# missing_count = df['gre_quantitative_reasoning'].isna().sum()

# # Step 2: Tune these proportions manually (no scaling)
# low_pct = 0.40
# mid_pct = 0.20
# high_pct = 0.40  # increased high values for mode control

# low_count = int(low_pct * missing_count)
# mid_count = int(mid_pct * missing_count)
# high_count = missing_count - low_count - mid_count

# # Step 3: Generate values directly
# low_vals = np.random.uniform(130, 140, low_count)
# mid_vals = np.random.uniform(140, 160, mid_count)
# high_vals = np.random.uniform(165, 170, high_count)

# # Step 4: Combine and round
# combined_vals = np.concatenate([low_vals, mid_vals, high_vals])
# rounded_vals = np.round(combined_vals)

# # Optional: prevent 130-mode trap
# rounded_vals[rounded_vals == 130] = 131

# # Step 5: Fill in
# df.loc[df['gre_quantitative_reasoning'].isna(), 'gre_quantitative_reasoning'] = rounded_vals


In [ ]:
# median_val = df[list_of_gre_columns[0]].median()
# mode_val = df[list_of_gre_columns[0]].mode()[0]  # mode() returns a Series
# mean_val = df[list_of_gre_columns[0]].mean()

# print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 154.0, Mode: 169.0, Mean: 150.21250186335405


In [3622]:
(df[list_of_gre_columns[0]].isna().sum())

0

In [3623]:
fig = px.histogram(valid_gre_quantitative_reasoning_profiles, x=list_of_gre_columns[1], nbins=100, range_x=[129,171],
                   title=f'Distribution of Valid {list_of_gre_columns[1]} Scores')
fig.show()

In [3624]:
median_val = df[list_of_gre_columns[1]].median()
mode_val = df[list_of_gre_columns[1]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[1]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 160.0, Mode: 162.0, Mean: 159.2960158644312


In [3625]:

# Count of missing
missing_count = df['gre_verbal_reasoning'].isna().sum()

# Step 1: Generate truncated normal values in 130–170
mean, std = 149, 8  # Approx based on visual and given stats
lower, upper = 130, 170
a, b = (lower - mean) / std, (upper - mean) / std
random_vals = truncnorm.rvs(a, b, loc=mean, scale=std, size=missing_count)

# Step 2: Adjust to match exact target mean
existing_vals = df['gre_verbal_reasoning'].dropna()
target_mean = 149
target_total = target_mean * (len(existing_vals) + missing_count)
required_sum = target_total - existing_vals.sum()

scaling_factor = required_sum / np.sum(random_vals)
adjusted_vals = np.clip(random_vals * scaling_factor, 130, 170)

# Step 3: Fill in the values
df.loc[df['gre_verbal_reasoning'].isna(), 'gre_verbal_reasoning'] = adjusted_vals.round()


In [3626]:
(df[list_of_gre_columns[1]]).isna().sum()

0

In [3629]:
median_val = df[list_of_gre_columns[1]].median()
mode_val = df[list_of_gre_columns[1]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[1]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 149.0, Mode: 148.0, Mean: 149.00150062111803


In [ ]:
df.to_excel("../data/processed/preprocessed_final.xlsx")

### Checking rows with values :
#### Below 130 
#### Above 340
#### Between 170 to 260

#### Checking for quantitative values

In [3630]:
df[list_of_gre_columns[0]] = pd.to_numeric(df[list_of_gre_columns[0]], errors='coerce')

In [3631]:
rows_above_340_gre_quantitative_reasoning = (
    (df[list_of_gre_columns[0]] > 340) |
    (df[list_of_gre_columns[0]] < 130) |
    ((df[list_of_gre_columns[0]] > 170) &
    (df[list_of_gre_columns[0]] < 340))  # This overlaps with < 130 and < 170
).sum()

print(f"Invalid rows for {list_of_gre_columns}: {rows_above_340_gre_quantitative_reasoning}")

Invalid rows for ['gre_quantitative_reasoning', 'gre_verbal_reasoning']: 1568


#### Checking for verbal values

In [3632]:
rows_above_340_gre_verbal_reasoning = (df[list_of_gre_columns[1]] >= 260).sum()
rows_above_340_gre_verbal_reasoning

8

In [3633]:
# df.loc[df[list_of_gre_columns[0] > 260]]

In [3634]:
gre_quantitative_reasoning_above_170_and_below_260 = df.loc[(df['gre_quantitative_reasoning'] > 170) & (df['gre_quantitative_reasoning'] < 260), 'gre_quantitative_reasoning']
len(gre_quantitative_reasoning_above_170_and_below_260)

12

In [3635]:
df = pd.to_numeric(df['gre_verbal_reasoning'])

### Checking valid GRE Verbal scores

In [3636]:
df['gre_score_verbal_max'] 

KeyError: 'gre_score_verbal_max'

In [ ]:
condition = (df['undergrad_gpa'] >= 130) & (df['undergrad_gpa'] < 171)

In [ ]:
df['gre_verbal'] = pd.to_numeric(df['gre_verbal'], errors = 'coerce')

In [ ]:
gre_score_general_min = df['gre_verbal'] >= 130
gre_score_general_max = df['gre_verbal'] <= 170

In [ ]:
df['analytical_writing'] = pd.to_numeric(df['analytical_writing'], errors = 'coerce')

In [ ]:
gre_analytical_writing_min = df['analytical_writing'] >= 0.0
gre_analytical_writing_max = df['analytical_writing'] >= 6.0

### Correlation between different variables and their visualisation

In [ ]:
# Convert relevant fields
df['decision_encoded'] = df['decision'].map({
    'Accepted': 1,
    'Rejected': 0,
    'Interview': 0.5,  # optional: if you want to include partial decision stages
    'Waitlisted': 0.25,
    'Other': None
})
